### Author: Juan Marcos Requena Gutiérrez
#### Objective: To create a deep learning model from scratch capable of distinguishing satellite images with four labels: cloudy, desert, green zone, and water. The goal of this first part is to train the model and calculate the accuracy on both the training and validation sets, aiming for reasonably good values. In the upcoming notebooks, we will explain how these accuracy values can be improved.

In [3]:
# Do it only one time
!pip install torch
!pip install torchvision

In [11]:
!nvidia-smi

Mon Mar 24 11:39:37 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.52                 Driver Version: 536.52       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P3              12W /  60W |      0MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
# Do it only one time
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 4.2 MB/s eta 0:09:49
     ---------------------------------------- 0.0/2.4 GB 4.0 MB/s eta 0:10:15
     ---------------------------------------- 0.0/2.4 GB 4.4 MB/s eta 0:09:12
     ---------------------------------------- 0.0/2.4 GB 4.6 MB/s eta 0:08:48
     ---------------------------------------- 0.0/2.4 GB 4.4 MB/s eta 0:09:17
     ---------------------------------------- 0.0/2.4 GB 4.6 MB/s eta 0:08:52
     ---------------------------------------- 0.0/2.4 GB 4.6 MB/s eta 0:08:49
     ---------------------------------------- 0.0/2.4 GB 4.3 MB/s eta 0:09:26
     ---------------------------------------- 0.0/2.4 GB 4.2 MB/s eta 0:09:43
     ---------------------------------------- 0.0/2.4 GB 4.1 MB/s eta 0:09:52
     ---------------------------------------- 0.0/2.4 GB 4.3 MB/s eta 0:09:26
     -------

In [280]:
!pip install utils


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13936 sha256=ae9154ab2cceb5e48715c204b17d3c00da13fbe45108a2a2926efb43a7bb6f32
  Stored in directory: c:\users\34651\appdata\local\pip\cache\wheels\b6\a1\81\1036477786ae0e17b522f6f5a838f9bc4288d1016fc5d0e1ec
Successfully built utils


In [138]:
torch._dynamo.config.suppress_errors = True

In [140]:
# Import the libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam
import utils

# Visualization tools
from torchvision import datasets, transforms
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

In [142]:
import logging
logging.getLogger("torch._dynamo").setLevel(logging.ERROR)

In [144]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [146]:
# Directory of the data
directory = r"C:\Users\34651\Desktop\MASTER\Modulo 9 (Deep Learning con CPU'S)\Challenge\data"

# Transform the images (resize, to tensor)
transform = transforms.Compose([
    transforms.Resize((32,32)), 
    transforms.ToTensor()
])

C:\Users\34651\anaconda3\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [148]:
# Load the images
data = datasets.ImageFolder(root = directory,  transform=transform)

In [150]:
# Define the train and validation dataset
train_size = int(0.8 * len(data))
val_size = len(data) - train_size

train_data, val_data = random_split(data, [train_size, val_size])

In [152]:
# Create the DataLoader 
batch_size = 32

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [154]:
# Show the size of the first batch of images
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break

torch.Size([32, 3, 32, 32]) torch.Size([32])


In [156]:
# Check that there are 32 labels, corresponding to each image in a batch
data_iter = iter(train_loader)
x_0, y_0 = next(data_iter)
y_0

tensor([1, 3, 0, 2, 0, 3, 0, 1, 3, 0, 1, 1, 1, 0, 0, 3, 1, 3, 2, 2, 2, 3, 0, 0,
        0, 0, 1, 0, 1, 3, 0, 2])

In [236]:
# Define the Convolutional Layer

class MyConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, dropout_p):
        kernel_size = 3
        super().__init__()

        self.model = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size, stride=1, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Dropout(dropout_p)
        )

    def forward(self, x):
        return self.model(x)

In [238]:
# Define the model

color_channel = 3 # RGB
N_CLASSES = 4
#in_features = 16384 # Para que las dimensiones tras las convolucionales sean correctas

model = nn.Sequential(
    MyConvBlock(color_channel, 8, 0.3),
    MyConvBlock(8, 16, 0.3),
    nn.Flatten(),
    nn.LazyLinear(256), # Automatically infers input features
    nn.Dropout(.5),
    nn.ReLU(),
    nn.Linear(256, N_CLASSES)      
)

In [240]:
# Chose the loss funtion and the optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

model = torch.compile(model.to(device))
model 

OptimizedModule(
  (_orig_mod): Sequential(
    (0): MyConvBlock(
      (model): Sequential(
        (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.3, inplace=False)
      )
    )
    (1): MyConvBlock(
      (model): Sequential(
        (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.3, inplace=False)
      )
    )
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): LazyLinear(in_features=0, out_features=256, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=4, bias=True)
  )
)

In [205]:
# Move the images to the gpu
for images, label in train_loader:
    images, labels = images.to(device), labels.to(device)

In [207]:
print(f"Model device: {next(model.parameters()).device}")
print(f"Data device: {images.device}")

Model device: cuda:0
Data device: cuda:0


#### Doing some Data Augmentation

In [242]:
random_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=0, translate=None, scale=(0.8, 1.2), shear=None),  # Aply zoom
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalice
])

In [244]:
# Transformation for validation
val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Only normalisation
])

In [246]:
def get_batch_accuracy(outputs, labels, N):
    _, predicted = outputs.max(1)
    correct = (predicted == labels).sum().item()
    return correct

#### Training the model

In [249]:
def train():
    loss = 0
    accuracy = 0

    train_N = len(train_loader.dataset)

    model.train()
    for x,y, in train_loader:
        x = x.float() # Transform into float32
        x,y = x.to(device), y.to(device)
        
        output = model(random_transforms(x))
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    accuracy = accuracy / train_N
    print('Train -Loss: {:.4f} Accuracy {:.4f}'.format(loss, accuracy))    

#### The validation dataset doesn't needd to be transformed

In [252]:
def validate():
    loss = 0
    accuracy = 0

    valid_N = len(val_loader.dataset)

    model.eval()
    with torch.no_grad():
        for x,y in val_loader:
            x,y = x.to(device), y.to(device)
            output = model(val_transforms(x))

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    accuracy = accuracy / valid_N
    print('Valid -Loss: {:.4f} Accuracy {:.4f}'.format(loss, accuracy))  

In [254]:
epochs = 10

for epoch in range (epochs):
    print('Epoch {}'.format(epoch))
    train()
    validate()

Epoch 0
Train -Loss: 109.3340 Accuracy 0.7425
Valid -Loss: 16.1881 Accuracy 0.7737
Epoch 1
Train -Loss: 63.9468 Accuracy 0.8035
Valid -Loss: 19.5033 Accuracy 0.7791
Epoch 2
Train -Loss: 60.5819 Accuracy 0.8139
Valid -Loss: 23.4469 Accuracy 0.7453
Epoch 3
Train -Loss: 58.2495 Accuracy 0.8204
Valid -Loss: 12.6405 Accuracy 0.8367
Epoch 4
Train -Loss: 55.0812 Accuracy 0.8397
Valid -Loss: 13.8178 Accuracy 0.8012
Epoch 5
Train -Loss: 59.3878 Accuracy 0.8235
Valid -Loss: 12.3809 Accuracy 0.8527
Epoch 6
Train -Loss: 56.5484 Accuracy 0.8266
Valid -Loss: 10.1905 Accuracy 0.9024
Epoch 7
Train -Loss: 57.4366 Accuracy 0.8357
Valid -Loss: 10.1418 Accuracy 0.8988
Epoch 8
Train -Loss: 53.1652 Accuracy 0.8348
Valid -Loss: 13.5183 Accuracy 0.7915
Epoch 9
Train -Loss: 53.8807 Accuracy 0.8368
Valid -Loss: 9.0298 Accuracy 0.8997
